# Inspect Simulation Parameters

Load simulation parameters from the output directory into a JSON file.

### Import Modules

In [1]:
import os
import sys
from pprint import pprint
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### List Simulation Directories

In [2]:
output_dir = "/scratch/users/jwakim/chromo_two_mark_phase_transition/output/full_factorial_batch_2"
all_sims = os.listdir(output_dir)
all_sims = [sim for sim in all_sims if sim.startswith("sim_")]
sim_inds = [int(sim.split("_")[-1]) for sim in all_sims]
all_sims = [sim for _, sim in sorted(zip(sim_inds, all_sims))]
sim_inds = np.sort(sim_inds)

sim_paths = {}
for sim in all_sims:
    sim_paths[sim] = f"{output_dir}/{sim}"

### Inspect Binder Parameters

In [3]:
binder_file = "binders"
binder_params = {}
for sim in all_sims:
    binder_path = f"{sim_paths[sim]}/{binder_file}"
    df_binder = pd.read_csv(binder_path, index_col="name")
    df_binder.dropna(axis=1, inplace=True)
    df_binder = df_binder.round(3)
    df_binder = df_binder.iloc[:, 1:]
    df_binder["cross_talk_field_energy_prefactor"] =\
        df_binder["cross_talk_field_energy_prefactor"].apply(
            lambda string_dict: {
                k: round(float(v), 3)
                for k, v in json.loads(string_dict.replace("\'", "\"")).items()
            }
        )
    binder_dict = df_binder.to_dict(orient="index")
    binder_params[sim] = binder_dict
pprint(binder_params)

{'sim_1': {'HP1': {'bind_energy_mod': -0.01,
                   'bind_energy_no_mod': 1.52,
                   'chemical_potential': -0.3,
                   'cross_talk_field_energy_prefactor': {'HP1': 0.0,
                                                         'PRC1': 1978379.047},
                   'cross_talk_interaction_energy': "{'PRC1': 0.75}",
                   'field_energy_prefactor': -6594596.824,
                   'interaction_energy': -5.0,
                   'interaction_energy_intranucleosome': -4.976,
                   'interaction_radius': 3.0,
                   'interaction_volume': 113.097,
                   'sites_per_bead': 2},
           'PRC1': {'bind_energy_mod': -0.01,
                    'bind_energy_no_mod': 1.52,
                    'chemical_potential': -0.3,
                    'cross_talk_field_energy_prefactor': {'HP1': 0.0,
                                                          'PRC1': 0.0},
                    'cross_talk_interaction_energy'

            'PRC1': {'bind_energy_mod': -0.01,
                     'bind_energy_no_mod': 1.52,
                     'chemical_potential': -0.5,
                     'cross_talk_field_energy_prefactor': {'HP1': 0.0,
                                                           'PRC1': 0.0},
                     'cross_talk_interaction_energy': '{}',
                     'field_energy_prefactor': -5935137.141,
                     'interaction_energy': -4.5,
                     'interaction_energy_intranucleosome': -4.478,
                     'interaction_radius': 3.0,
                     'interaction_volume': 113.097,
                     'sites_per_bead': 2}},
 'sim_52': {'HP1': {'bind_energy_mod': -0.01,
                    'bind_energy_no_mod': 1.52,
                    'chemical_potential': -0.5,
                    'cross_talk_field_energy_prefactor': {'HP1': 0.0,
                                                          'PRC1': 659459.682},
                    'cross_talk_interac

### Pair Simulations

Match simulations with matching binder parameters.

In [4]:
matching_sims = {}
for sim in all_sims:
    binder_dict = binder_params[sim]
    cp_HP1 = round(binder_dict["HP1"]["chemical_potential"], 3)
    cp_PRC1 = round(binder_dict["PRC1"]["chemical_potential"], 3)
    self_HP1 = round(binder_dict["HP1"]["interaction_energy"], 3)
    self_PRC1 = round(binder_dict["PRC1"]["interaction_energy"], 3)
    cross = round(
        json.loads(
            binder_dict["HP1"]["cross_talk_interaction_energy"].replace("\'", "\"")
        )["PRC1"], 3
    )
    params = f"{cp_HP1};{cp_PRC1};{self_HP1};{self_PRC1};{cross}"
    if params not in matching_sims.keys():
        matching_sims[params] = [sim]
    else:
        matching_sims[params].append(sim)

pprint(matching_sims)

{'-0.3;-0.3;-4.5;-4.5;0.25': ['sim_7', 'sim_38'],
 '-0.3;-0.3;-4.5;-4.5;0.75': ['sim_6', 'sim_48'],
 '-0.3;-0.3;-4.5;-5.0;0.25': ['sim_8', 'sim_34'],
 '-0.3;-0.3;-4.5;-5.0;0.75': ['sim_12', 'sim_40'],
 '-0.3;-0.3;-5.0;-4.5;0.25': ['sim_13', 'sim_36'],
 '-0.3;-0.3;-5.0;-4.5;0.75': ['sim_10', 'sim_43'],
 '-0.3;-0.3;-5.0;-5.0;0.25': ['sim_9', 'sim_33'],
 '-0.3;-0.3;-5.0;-5.0;0.75': ['sim_1', 'sim_35'],
 '-0.3;-0.5;-4.5;-4.5;0.25': ['sim_16', 'sim_55'],
 '-0.3;-0.5;-4.5;-4.5;0.75': ['sim_26', 'sim_63'],
 '-0.3;-0.5;-4.5;-5.0;0.25': ['sim_21', 'sim_47'],
 '-0.3;-0.5;-4.5;-5.0;0.75': ['sim_14', 'sim_58'],
 '-0.3;-0.5;-5.0;-4.5;0.25': ['sim_24', 'sim_53'],
 '-0.3;-0.5;-5.0;-4.5;0.75': ['sim_11', 'sim_51'],
 '-0.3;-0.5;-5.0;-5.0;0.25': ['sim_4', 'sim_39'],
 '-0.3;-0.5;-5.0;-5.0;0.75': ['sim_2', 'sim_37'],
 '-0.5;-0.3;-4.5;-4.5;0.25': ['sim_28', 'sim_60'],
 '-0.5;-0.3;-4.5;-4.5;0.75': ['sim_25', 'sim_45'],
 '-0.5;-0.3;-4.5;-5.0;0.25': ['sim_30', 'sim_44'],
 '-0.5;-0.3;-4.5;-5.0;0.75': ['sim_3',

### Load Field Parameters

In [5]:
def convert_to_numeric(init_val):
    try:
        val = float(init_val)
        if val.is_integer():
            val = int(val)
        else:
            val = round(val, 3)
        return val
    except:
        return init_val

field_file = "UniformDensityField"
field_params = {}
for sim in all_sims:
    field_path = f"{sim_paths[sim]}/{field_file}"
    df_field = pd.read_csv(field_path, header=None, names=("Label", "Value"), index_col=0)
    df_field["Value"] = df_field["Value"].apply(lambda val: convert_to_numeric(val))
    field_dict = df_field.to_dict(orient="dict")["Value"]
    field_params[sim] = field_dict
pprint(field_params)

{'sim_1': {'Chr_CG': 'polymer',
           'HP1': 'binder',
           'PRC1': 'binder',
           'assume_fully_accessible': 'True',
           'chi': 1,
           'confine_length': 364.932,
           'confine_type': 'Spherical',
           'fast_field': 'True',
           'nx': 27,
           'ny': 27,
           'nz': 27,
           'vf_limit': 0.5,
           'x_width': 771.429,
           'y_width': 771.429,
           'z_width': 771.429},
 'sim_10': {'Chr_CG': 'polymer',
            'HP1': 'binder',
            'PRC1': 'binder',
            'assume_fully_accessible': 'True',
            'chi': 1,
            'confine_length': 364.932,
            'confine_type': 'Spherical',
            'fast_field': 'True',
            'nx': 27,
            'ny': 27,
            'nz': 27,
            'vf_limit': 0.5,
            'x_width': 771.429,
            'y_width': 771.429,
            'z_width': 771.429},
 'sim_11': {'Chr_CG': 'polymer',
            'HP1': 'binder',
            'PRC1':

### Create JSON Output of Simulation Parameters

In [6]:
all_params = {}
for sim in all_sims:
    param_dict = {
        "binder_parameters": binder_params[sim],
        "field_parameters": field_params[sim]
    }
    all_params[sim] = param_dict
pprint(all_params)

{'sim_1': {'binder_parameters': {'HP1': {'bind_energy_mod': -0.01,
                                         'bind_energy_no_mod': 1.52,
                                         'chemical_potential': -0.3,
                                         'cross_talk_field_energy_prefactor': {'HP1': 0.0,
                                                                               'PRC1': 1978379.047},
                                         'cross_talk_interaction_energy': "{'PRC1': "
                                                                          '0.75}',
                                         'field_energy_prefactor': -6594596.824,
                                         'interaction_energy': -5.0,
                                         'interaction_energy_intranucleosome': -4.976,
                                         'interaction_radius': 3.0,
                                         'interaction_volume': 113.097,
                                         'sites_per_bead'

 'sim_20': {'binder_parameters': {'HP1': {'bind_energy_mod': -0.01,
                                          'bind_energy_no_mod': 1.52,
                                          'chemical_potential': -0.5,
                                          'cross_talk_field_energy_prefactor': {'HP1': 0.0,
                                                                                'PRC1': 659459.682},
                                          'cross_talk_interaction_energy': "{'PRC1': "
                                                                           '0.25}',
                                          'field_energy_prefactor': -5935137.141,
                                          'interaction_energy': -4.5,
                                          'interaction_energy_intranucleosome': -4.478,
                                          'interaction_radius': 3.0,
                                          'interaction_volume': 113.097,
                                          'sit

                                          'interaction_energy': -5.0,
                                          'interaction_energy_intranucleosome': -4.976,
                                          'interaction_radius': 3.0,
                                          'interaction_volume': 113.097,
                                          'sites_per_bead': 2}},
           'field_parameters': {'Chr_CG': 'polymer',
                                'HP1': 'binder',
                                'PRC1': 'binder',
                                'assume_fully_accessible': 'True',
                                'chi': 1,
                                'confine_length': 364.932,
                                'confine_type': 'Spherical',
                                'fast_field': 'True',
                                'nx': 27,
                                'ny': 27,
                                'nz': 27,
                                'vf_limit': 0.5,
                         

            'field_parameters': {'Chr_refine': 'polymer',
                                 'HP1': 'binder',
                                 'PRC1': 'binder',
                                 'assume_fully_accessible': 'True',
                                 'chi': 1,
                                 'confine_length': 899.993,
                                 'confine_type': 'Spherical',
                                 'fast_field': 'True',
                                 'nx': 68,
                                 'ny': 68,
                                 'nz': 68,
                                 'vf_limit': 0.5,
                                 'x_width': 1942.857,
                                 'y_width': 1942.857,
                                 'z_width': 1942.857}},
 'sim_55': {'binder_parameters': {'HP1': {'bind_energy_mod': -0.01,
                                          'bind_energy_no_mod': 1.52,
                                          'chemical_potential': -0.3,
  

In [7]:
params_json = json.dumps(all_params, indent=4)
print(params_json)

{
    "sim_1": {
        "binder_parameters": {
            "HP1": {
                "sites_per_bead": 2,
                "interaction_energy": -5.0,
                "chemical_potential": -0.3,
                "bind_energy_mod": -0.01,
                "bind_energy_no_mod": 1.52,
                "cross_talk_field_energy_prefactor": {
                    "HP1": 0.0,
                    "PRC1": 1978379.047
                },
                "cross_talk_interaction_energy": "{'PRC1': 0.75}",
                "field_energy_prefactor": -6594596.824,
                "interaction_energy_intranucleosome": -4.976,
                "interaction_radius": 3.0,
                "interaction_volume": 113.097
            },
            "PRC1": {
                "sites_per_bead": 2,
                "interaction_energy": -5.0,
                "chemical_potential": -0.3,
                "bind_energy_mod": -0.01,
                "bind_energy_no_mod": 1.52,
                "cross_talk_field_energy_prefactor":